In [1]:
library(lme4)
library(sjPlot)
library(Hmisc)

library(gridExtra)
library(ggplot2)


# custom style for tab models
css = list(
    css.thead = 'border-top: double; text-align:center; font-style:normal; font-weight:bold; padding:0.1cm;',
    css.tdata = 'padding:0.05cm; padding-left:0.2cm; padding-right:0.2cm; text-align:left; vertical-align:top;',
    css.depvarhead = 'text-align:center; border-bottom:1px solid; font-style:italic; font-weight:bold',
    css.summary = 'padding-top:0.05cm; padding-bottom:0.05cm;'
  )

Loading required package: Matrix

Loading required package: lattice

Loading required package: survival

Loading required package: Formula

Loading required package: ggplot2


Attaching package: ‘Hmisc’


The following objects are masked from ‘package:base’:

    format.pval, units




# Read Data

In [2]:
wtdata <- read.csv("data/dataset_A_linked.csv")

# Scale and center variables

In [3]:
# scale and center the election periods
wtdata$is_before_elections <- scale(wtdata$election_period_cat == 'A', scale=T)
wtdata$is_during_elections <- scale(wtdata$election_period_cat == 'B', scale=T)
wtdata$is_after_elections <- scale(wtdata$election_period_cat == 'C', scale=T)

# scaling the rest of the variables
wtdata$rank <- scale(wtdata$rank, scale=T)
# invert this value so that the odds ratio, so it the scaled scores
# match the intuition, i.e. the 1st result should have a higher (positive)
# value. The odds ratio are then easier to interpret.
wtdata$inv_rank <- -1 * wtdata$rank

# scale and center the representativeness
wtdata$news_representativeness <- scale(wtdata$news_representativeness, scale=T)

# scale and center engagement variables
wtdata$engagement_TD_B <- scale(wtdata$engagement_TD_B, scale=T)
wtdata$engagement_TD_C <- scale(wtdata$engagement_TD_C, scale=T)
wtdata$engagement_TD_D <- scale(wtdata$engagement_TD_D, scale=T)

# scale and center engagement variables
wtdata$visits_TD_B <- scale(wtdata$visits_TD_B, scale=T)
wtdata$visits_TD_C <- scale(wtdata$visits_TD_C, scale=T)
wtdata$visits_TD_D <- scale(wtdata$visits_TD_D, scale=T)

# scale and center engagement variables
wtdata$allnews_visits_TD_B <- scale(wtdata$allnews_visits_TD_B, scale=T)
wtdata$allnews_visits_TD_C <- scale(wtdata$allnews_visits_TD_C, scale=T)
wtdata$allnews_visits_TD_D <- scale(wtdata$allnews_visits_TD_D, scale=T)

# convert to factors the within subjects
wtdata$search_page <- as.factor(wtdata$search_page)
wtdata$participant <- as.factor(wtdata$participant)
wtdata$coredomain <- as.factor(wtdata$coredomain)

# Dataset B: engagement

In [4]:
# time dependent
td_model_B <- glmer(is_selected_result ~ 

                            # main factors
                              visits_TD_B
                            * inv_rank
                            * news_representativeness 
                    
                            # control by total news visits
                            + allnews_visits_TD_B

                            # control for demographics
                            + is_female
                            + is_diverse
                            + is_west_germany
                            + age
                            + education
                            + political_alignment

                            # control for election day
                            + is_during_elections
                            + is_after_elections

                            # within subject controls
                            + (1|search_page)
                            + (1|coredomain)
                            + (1|participant)

                            , data=wtdata, family=binomial)

tab_model(td_model_B, wrap.labels=F, CSS=css, file = "statistical_tests/absolute_visits/td_model_B.html")


boundary (singular) fit: see help('isSingular')



# Dataset C: Google associated engagement

In [5]:
# time independent
ti_model_C <- glmer(is_selected_result ~ 

                            # main factors
                              visits_TD_C
                            * inv_rank
                            * news_representativeness 
                    
                            # control by total news visits
                            + allnews_visits_TD_C

                            # control for demographics
                            + is_female
                            + is_diverse
                            + is_west_germany
                            + age
                            + education
                            + political_alignment

                            # control for election day
                            + is_during_elections
                            + is_after_elections

                            # within subject controls
                            + (1|search_page)
                            + (1|coredomain)
                            + (1|participant)

                            , data=wtdata, family=binomial)

tab_model(ti_model_C, wrap.labels=F, CSS=css, file = "statistical_tests/absolute_visits/td_model_C.html")


boundary (singular) fit: see help('isSingular')



# Dataset D: non Google associated engagement

In [6]:
# time independent
ti_model_D <- glmer(is_selected_result ~ 

                            # main factors
                              visits_TD_D
                            * inv_rank
                            * news_representativeness 
                    
                            # control by total news visits
                            + allnews_visits_TD_D
                    
                            # control for demographics
                            + is_female
                            + is_diverse
                            + is_west_germany
                            + age
                            + education
                            + political_alignment

                            # control for election day
                            + is_during_elections
                            + is_after_elections

                            # within subject controls
                            + (1|search_page)
                            + (1|coredomain)
                            + (1|participant)

                            , data=wtdata, family=binomial)

tab_model(ti_model_D, wrap.labels=F, CSS=css, file = "statistical_tests/absolute_visits/td_model_D.html")


boundary (singular) fit: see help('isSingular')

